# Installing keras-tuner library for performing HyperParameter tuning: 

In [ ]:
!pip install keras-tuner

# Importing Libraries:

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [3]:
print(tf.__version__)

2.4.0


# Loading Fashion_Mnist Dataset:

In [4]:
dset = keras.datasets.fashion_mnist
dset

<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.6/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

# Splitting dset into train_dset and test_dset:

In [5]:
(X_train, y_train), (X_test, y_test) = dset.load_data()

In [6]:
X_train, y_train

(array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],

In [7]:
X_test, y_test

(array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],

# Image Data Preprocessing:

In [8]:
# scaling images to fall in range of (0 to 255):
X_train = X_train/255.0
X_test = X_test/255.0

# reshaping images:
X_train = X_train.reshape(len(X_train), 28, 28, 1)
X_test = X_test.reshape(len(X_test), 28, 28, 1)

# Define fxn build_cnn for building diff cnns:

In [9]:
def build_cnn(hp):

    cnn = keras.Sequential()
    
    # 1st conv2D layer:
    cnn.add(keras.layers.Conv2D(
        filters=hp.Int(
            'Conv2D_L1_Filter', min_value=32, max_value=128, step=16 ),
        kernel_size=hp.Choice(
            'kernel_01', values=[3,5] ),
        activation='relu',
        input_shape=(28,28,1),
    ))

    # 2nd conv2D layer:
    cnn.add(keras.layers.Conv2D(
        filters=hp.Int(
            'Conv2D_L2_Filter', min_value=32, max_value=64, step=16 ),
        kernel_size=hp.Choice(
            'kernel_02', values=[3,5] ),
        activation='relu'
    ))

    # Flatten Layer:
    cnn.add(keras.layers.Flatten())

    # 1st Dense Layer:
    cnn.add(keras.layers.Dense(
        hp.Int(
            'Dense_01',
            min_value=32,
            max_value=128,
        ),
        activation='relu'
    ))

    # 2nd Dense Layer / Output Layer:
    cnn.add(keras.layers.Dense(
        10, # 10 neurons for 10 fashion items/labels in Fashion_mnist dset.
        activation='softmax'
        ))
    
    
    # compiling the cnn model.
    cnn.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return cnn

# Create tuner using keras's RandomSearch that can try out diff hyperParameter combinations 

In [10]:
# build the tuner that can try out diff combinations of hyperParameters:
from kerastuner import RandomSearch

tuner = RandomSearch(
    build_cnn, # fxn_name
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name='fashion_mnist')

INFO:tensorflow:Reloading Oracle from existing project output/fashion_mnist/oracle.json
INFO:tensorflow:Reloading Tuner from output/fashion_mnist/tuner0.json


In [11]:
# print the summary of the search space:
tuner.search_space_summary()

Search space summary
Default search space size: 6
Conv2D_L1_Filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
kernel_01 (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
Conv2D_L2_Filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
kernel_02 (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
Dense_01 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


# Perform the search() on tuner:

In [12]:
# search for the best hyperParameters:

tuner.search(X_train, y_train,
             epochs=5,
             validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


# Find the best performing CNN:

In [13]:
# print the top 1st performing best model:
top_cnn = tuner.get_best_models(num_models=1)[0]
print(top_cnn)


# get tuner-results summary:
tuner.results_summary()

Results summary
Results in output/fashion_mnist
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Conv2D_L1_Filter: 32
kernel_01: 3
Conv2D_L2_Filter: 64
kernel_02: 5
Dense_01: 51
learning_rate: 0.001
Score: 0.9139999747276306
Trial summary
Hyperparameters:
Conv2D_L1_Filter: 112
kernel_01: 3
Conv2D_L2_Filter: 32
kernel_02: 3
Dense_01: 44
learning_rate: 0.0001
Score: 0.8944166898727417
Trial summary
Hyperparameters:
Conv2D_L1_Filter: 64
kernel_01: 3
Conv2D_L2_Filter: 64
kernel_02: 3
Dense_01: 51
learning_rate: 0.0001
Score: 0.893916666507721
Trial summary
Hyperparameters:
Conv2D_L1_Filter: 64
kernel_01: 5
Conv2D_L2_Filter: 48
kernel_02: 5
Dense_01: 52
learning_rate: 0.0001
Score: 0.8930000066757202
Trial summary
Hyperparameters:
Conv2D_L1_Filter: 32
kernel_01: 3
Conv2D_L2_Filter: 48
kernel_02: 3
Dense_01: 59
learning_rate: 0.01
Score: 0.8832499980926514


# Lets fit the train_dset to the best performing CNN:

In [14]:
top_cnn.fit(X_train, y_train, epochs=20, validation_split=0.1, initial_epoch=5)

Epoch 6/20
1688/1688 [==============================] - 9s 4ms/step - loss: 0.1277 - accuracy: 0.9536 - val_loss: 0.2800 - val_accuracy: 0.9163
Epoch 7/20
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0849 - accuracy: 0.9694 - val_loss: 0.2929 - val_accuracy: 0.9142
Epoch 8/20
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0567 - accuracy: 0.9790 - val_loss: 0.3794 - val_accuracy: 0.9115
Epoch 9/20
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0416 - accuracy: 0.9855 - val_loss: 0.3739 - val_accuracy: 0.9123
Epoch 10/20
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0311 - accuracy: 0.9895 - val_loss: 0.4614 - val_accuracy: 0.9115
Epoch 11/20
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0245 - accuracy: 0.9912 - val_loss: 0.4441 - val_accuracy: 0.9098
Epoch 12/20
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0245 - accuracy: 0.9913 - val_loss: 0.4819 - val_accura

# Make a new Prediction:

In [15]:
from keras.preprocessing import image

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

img_test = X_test[:5]
y_pred = top_cnn.predict_classes(img_test)
# print(y_pred)
print("Cnn's predicted Labels: \n",np.array(class_names)[y_pred])

print("True Labels: \n",np.array(class_names)[y_test[:5]])

Cnn's predicted Labels: 
 ['Ankle boot' 'Pullover' 'Trouser' 'Trouser' 'Shirt']
True Labels: 
 ['Ankle boot' 'Pullover' 'Trouser' 'Trouser' 'Shirt']


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
